<a href="https://colab.research.google.com/github/kareemrasheed89/DataQuestVisualization-Proj/blob/master/LLama_RAG_WIth_HuggingFaceEmb_ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Building RAG solution with GROQ and HuggingFaceEmbediing and Levaraging LangChain

----------------------------------------------
Chroma is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0. View the full docs of Chroma at this page, and find the API reference for the LangChain integration at this page.

In [1]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers unstructured==0.15.0 unstructured[pdf]==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

# Section A
------------
Import all relevant libraries from langchain and langchain community

In [11]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')

In [2]:
#**Import all relevant libraries*/
from langchain.document_loaders import UnstructuredFileLoader, OnlinePDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

### Section B
-------------
Load the knoweledge documents and chunk it into 1000 chunk size and 50 overlap

In [3]:
loader= UnstructuredFileLoader("/content/XGBoost.pdf")
Documents=loader.load()

In [6]:
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
texts=text_splitter.split_documents(Documents)
type(texts)


list

In [7]:
len(texts)

9

In [8]:
texts[:3]

[Document(metadata={'source': '/content/XGBoost.pdf'}, page_content='How to Develop Your First XGBoost Model in Python\n\nmachinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn\n\nXGBoost is an implementation of gradient boosted decision trees designed for speed and performance that is dominative competitive machine learning.\n\nIn this post you will discover how you can install and create your first XGBoost model in Python.\n\nAfter reading this post you will know:\n\nHow to install XGBoost on your system for use in Python. How to prepare data and train your first XGBoost model. How to make predictions using your XGBoost model.\n\nLet’s get started.\n\nHow to Develop Your First XGBoost Model in Python with scikit-learn Photo by Justin Henry, some rights reserved.\n\nTutorial Overview\n\nThis tutorial is broken down into the following 6 sections:\n\n1/6'),
 Document(metadata={'source': '/content/XGBoost.pdf'}, page_content='1/6\n\n1. Install XGBoost for use with Pyt

### Section C
---------------
Embedd the documents chunks with HuggingFaceEmbedding model for easy retrieval for retreival model.


In [4]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-4-0ce1537519dd>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
persist_directory="/content/db"
vectordb=Chroma.from_documents(texts,embedding=embeddings,persist_directory=persist_directory)
vectordb.persist()

<ipython-input-9-c75e734f443a>:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [14]:
retriever=vectordb.as_retriever()

In [12]:
llm=ChatGroq(model="llama-3.1-70b-versatile",
             temperature=0  )

In [15]:
qa_chain=RetrievalQA.from_chain_type(llm=llm,
                                     chain_type="stuff",
                                     retriever=retriever, return_source_documents=True)

In [16]:
query="What is XGboost?"
response=qa_chain({"query":query})
print(response["result"])

<ipython-input-16-52a0196d0910>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  response=qa_chain({"query":query})


XGBoost is an implementation of gradient boosted decision trees designed for speed and performance that is dominative competitive machine learning.


In [17]:
print(response['source_documents'][0].metadata["source"])

/content/XGBoost.pdf


In [18]:
query="What is XGboost?"
response=qa_chain({"query":query})
print(response["result"])
print(response['source_documents'][0].metadata["source"])

XGBoost is an implementation of gradient boosted decision trees designed for speed and performance that is dominative competitive machine learning.
/content/XGBoost.pdf
